# architecture-29-02

## What's new:

1- Extract data rows with a formula instead of manually extracting them.


In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


2025-10-06 15:58:41.526082: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-06 15:58:41.537175: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759753721.548897    9127 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759753721.552652    9127 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-06 15:58:41.564746: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Load and Scaling Features

df = pd.read_csv('XAGUSD-197001010000--H1-rates.csv', sep='\t')

# Select features to scale
features = ['<OPEN>', '<HIGH>', '<LOW>', '<CLOSE>', '<TICKVOL>']

# Apply MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df[features])

In [3]:
close_scaler = MinMaxScaler()
scaled_close = close_scaler.fit_transform(df[['<CLOSE>']])

In [4]:
# Prepare sequences
def create_sequences(features, target, window, horizon):
    X, y = [], []
    for i in range(window, len(features) - horizon):
        X.append(features[i - window:i])
        y.append(target[i:i + horizon])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled, scaled_close, 60, 10)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build basic LSTM model
model = Sequential([
    LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(10)
])
model.compile(optimizer='adam', loss='mse', metrics=['mae'])



2025-10-06 15:58:54.309475: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-10-06 15:58:54.309495: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: saeed-VivoBook-ASUSLaptop-X513EPN-R528EP
2025-10-06 15:58:54.309499: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: saeed-VivoBook-ASUSLaptop-X513EPN-R528EP
2025-10-06 15:58:54.309578: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 570.172.8
2025-10-06 15:58:54.309589: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 570.172.8
2025-10-06 15:58:54.309592: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 570.172.8
/home/saeed/repositories/machine-learning/fore

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    verbose=1,
    validation_split=0.2,
)

In [5]:
# If X.dtype or y.dtype is float64 or float128, convert them (next step). If NaN/Inf is True, fix those entries.

import numpy as np, tensorflow as tf
print("tf:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))
print("X dtype / y dtype:", X.dtype, y.dtype)
print("X shape / y shape:", X.shape, y.shape)
print("Any NaN/Inf in X/y:", np.any(np.isnan(X)), np.any(np.isinf(X)), np.any(np.isnan(y)), np.any(np.isinf(y)))

tf: 2.18.0
GPUs: []
X dtype / y dtype: float64 float64
X shape / y shape: (99930, 60, 5) (99930, 10, 1)
Any NaN/Inf in X/y: False False False False


In [6]:
# If tf.config.list_physical_devices('GPU') is empty, TF can't see a GPU. If GPUs exist but CUDA/CUDNN versions are missing or incompatible, that points to driver/stack issues.

print("TF version:", tf.__version__)
print("GPUs visible:", tf.config.list_physical_devices('GPU'))
print("CUDA/CUDNN build info:", tf.sysconfig.get_build_info().get('cuda_version', None),
      tf.sysconfig.get_build_info().get('cudnn_version', None))

TF version: 2.18.0
GPUs visible: []
CUDA/CUDNN build info: 12.8 9
